## Reactive Functions & Expressions

In the [Dependencies and Watchers](Dependencies_and_Watchers.ipynb) guide we discovered how to express dependencies and write callbacks that are invoked when parameter values change. This imperative style of expressing dynamic behavior is powerful but can also end up resulting in complex code that is hard to read and maintain. Therefore Param 2.0 introduces a set of primitives that make it possible to express complex dependencies in a declarative form resulting in reactive functions and expressions that are re-evaluated automatically when required.

The reactive programming model is one you might be familiar with from Excel, where formulas can reference cells or ranges and dynamically (or rather reactively) recompute when the inputs to a formula changes. In Param, Parameter objects represent the inputs or references in a formula and reactive expressions replace the concept of a formula. Additionally `param.bind` allows the creation of a reactive function with arbitrary inputs.

## Reactive Functions

Let us start by creating a `Parameterized` class with some parameters, here named `a` and `b`. Additionally we import `param.ipython` to allow reactive components to display correctly and re-render themselves when the inputs change.

In [ ]:
import param
import param.ipython

class Parameters(param.Parameterized):
    
    a = param.Number(default=1)

    b = param.Number()

Now we will create a reactive function using `param.bind`. We start by declaring a function that adds two parameters and then bind parameters `a` and `b` to that function:

In [ ]:
def add(a, b):
    return a+b

p = Parameters()

reactive_add = param.bind(add, p.param.a, p.param.b)

reactive_add

You can see that this reactive function renders the output and when we update one of the inputs to the function the output will reflect the updated result.

In [ ]:
p.a = 5

We can also call the reactive function to return the current result:

In [ ]:
reactive_add()

Note that if you only partially bind the required arguments the function will not be automatically evaluated:

In [ ]:
param.bind(add, p.param.a)

## Reactive Expressions

While reactive functions are very useful and allow writing arbitrary logic they still require writing the actual functions which result in more indirection and less readability. With a `reactive` expression you can wrap any object or parameter value and apply operations on it, as if you are working with the actual object. In other words, it acts as a proxy of the underlying value and supports (almost) all operations that can be performed with the original object.

In order to avoid any clashes between the namespace of the reactive expression and the object it is wrapping, it has a special name space called `.rx`, that contains special methods to work with the expression.  

### Using Parameters

As an example we can create reactive proxies for the `a` and `b` parameters and add them together:

In [ ]:
p = Parameters()

expr = p.param.a.reactive() + p.param.b.reactive() + 3

expr

The resulting reactive expression now reflects the result of this operation and will update automatically when one of the inputs to the operation changes, e.g. if we update parameter `a`:

In [ ]:
p.a = 5

### Resolving the expression

To resolve the current value of the expression we can call `.rx.resolve()`:

In [ ]:
expr.rx.resolve()

### Using literal objects as inputs

Note that the input to a reactive expression can be anything, e.g. we can create a `reactive` object from a static, literal value, such as a string:

In [ ]:
string_template = param.reactive('Hello {name}!')

string_template

The `reactive` object now acts like a string so we can use the `.format` method to fill in the `string_template`:

In [ ]:
name = param.reactive('world')

str_expr = string_template.format(name=name)

str_expr

### Setting the input value

To update the input to a `reactive` object we can use the `.rx.set()` method:

In [ ]:
name.rx.set('there')

str_expr.rx.resolve()

### Limitations

A `reactive` proxy behaves like the underlying object it is wrapping, but only to the extent that Python allows. Certain operations cannot be implemented, e.g. Python will not allow the `len` operation to return anything but a integer and the `is` statement always checks the immediate identity of its two operands. Therefore `reactive` implements certain operations as special methods that exist on the `.rx` namespace, these include:

- `.bool()`: Tests the truthiness of the object
- `.is_()`: Tests the object identity against another object.
- `.is_not()`: Tests the object identity against another object.
- `.len()`: The length of the expression
- `.pipe()`: Allows applying an arbitrary function with static or reactive arguments to the object.
- `.when()`: Generates a new expression that only updates when the provided dependency updates.